## CNN timeseries class.

In [ ]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

aa = pd.read_csv(root_url + "FordA_TRAIN.tsv",delimiter='\t',header=None)

In [ ]:
aa

In [ ]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)


root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

In [ ]:
x_train.shape

In [ ]:
plt.figure()
plt.plot(x_train[103,:])

In [ ]:
classes = np.unique(np.concatenate((y_train, y_test), axis=0))

plt.figure()
for c in classes:
    c_x_train = x_train[y_train == c]
    plt.plot(c_x_train[0], label="class " + str(c))
plt.legend(loc="best")
plt.show()
plt.close()

In [ ]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

In [ ]:
num_classes = len(np.unique(y_train))
print(num_classes)

In [ ]:
idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]


In [ ]:
y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

### Get data from GP

In [1]:
import pandas as pd
import numpy as np

from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
data_gp = pd.read_csv('../data/gp_training_data.csv')

In [ ]:
data_gp

In [ ]:
n_obj = len(data_gp['object_id'].unique())
n_obs = len(data_gp[(data_gp['object_id'] == 615) & (data_gp['passband'] == 0)])

In [ ]:
x_train = np.ndarray((n_obj,n_obs,6))

In [ ]:
unique_obj = data_gp['object_id'].unique()
y_train = []
for i, obj_id in enumerate(unique_obj):
    if i%1000 == 0: print(i)
    y_train.append(obj_id)
    for z in range(6):
        lc_temp = data_gp[(data_gp['object_id'] == obj_id) & (data_gp['passband'] == z)]
        x_train[i,:,z] = lc_temp['flux']
    
    
    

In [ ]:
x_train.nbytes/1e9

In [ ]:
np.save('train_format_gp.npy',x_train)

## Data preparation (after saving x_train)

In [2]:
meta_ = pd.read_csv('../data/PLAsTiCC/training_set_metadata.csv')
data_gp = pd.read_csv('../data/gp_training_data.csv')

In [3]:
unique_obj = data_gp['object_id'].unique()

In [4]:
unique_obj

array([      615,       713,       730, ..., 130762946, 130772921,
       130779836])

In [ ]:
category_mapping[67]

In [6]:
category_mapping = {90: 'SN1a', 67: 'SN1a-91bg', 52: 'SN1ax', 42: 'SN2', 62: 'SN1bc', 95: 'SLSN1', 15: 'TDE', 64: 'KN', 88: 'AGN', 92: 'RRL', 65: 'M-dwarf', 16: 'EB', 53: 'Mira', 6: 'Microlens'}

In [11]:
y_train = []
for i,obj_id in enumerate(unique_obj):
    class_ = meta_[meta_['object_id'] == obj_id]['target'].values[0]
    
    if category_mapping[class_] == 'AGN':
        y_train.append(1)
    else:
        y_train.append(0)
    
y_train = np.array(y_train)
np.save('data/train_format_gp_labels.npy',y_train)

Simeon start from here with the Ferrari

In [ ]:
import pandas as pd
import numpy as np

from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
x_train = np.load('data/train_format_gp.npy')
y_train = np.load('data/train_format_gp_labels.npy')
print(x_train.shape)


In [ ]:
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)

num_classes = len(np.unique(y_train))


model = make_model(input_shape=x_train.shape[1:])
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
epochs = 100
batch_size = 32

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

In [ ]:
model = keras.models.load_model("best_model.h5")

test_loss, test_acc = model.evaluate(x_test, y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

In [ ]:
metric = "sparse_categorical_accuracy"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.show()
plt.close()